Importing data generator and training

In [4]:
import sys, os
sys.path.append(os.getcwd())
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import data_gen1

np.random.seed(69)
# torch.device('cuda' if torch.cuda.is_available else 'cpu')
device = torch.device('cpu')

In [21]:
def create_data(num_points, point_size=1000, min_var0=0.1, max_var0=10, min_var1=0.1, max_var1=10, dim=1):
    x0_var = np.random.uniform(min_var0, max_var0, num_points)
    x1_var = np.random.uniform(min_var1, max_var1, num_points)
    data = np.zeros([num_points, point_size, 2, dim])
    label = np.zeros([num_points])

    for i in range(num_points):
        data[i], label[i] = data_gen1.gen_list(
            point_size, x0_var[i], x1_var[i], dim)

    return data, label

def create_data2(realisations, xy_len=1000, min_var0=0.1, max_var0=10, min_var1=0.1, max_var1=10, dim=1):
    x0_var = np.random.uniform(min_var0, max_var0, realisations)
    x1_var = np.random.uniform(min_var1, max_var1, realisations)
    data = np.zeros((realisations, xy_len, 2))
    label = np.zeros([realisations])

    for i in range(realisations):
        _test, label[i] = data_gen1.gen_list(
            xy_len, x0_var[i], x1_var[i], dim)
        x, y = _test[:, 0, :], _test[:, 1, :]

        data[i] = np.hstack((x, y))

    return data, label

data, label = create_data2(2, 2)
data, label

(array([[[-1.17437157, -0.10885711],
         [ 3.50118478,  3.87670562]],
 
        [[-1.36466648, -0.54565739],
         [ 3.45002393,  3.8294597 ]]]),
 array([0.5450571 , 0.77598356]))

In [36]:
# Test data import
class Article_nn(nn.Module):
    def __init__(self, input_size):
        super(Article_nn, self).__init__()
        self.l1 = nn.Linear(input_size*2, 64)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(64, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.sig(self.l2(out))
        return out


class NumbersDataset(Dataset):
    def __init__(self, num_points, point_size):
        self.samples, self.labels = create_data2(num_points, point_size)
        self.samples = torch.from_numpy(self.samples).to(torch.float32)
        self.labels = torch.from_numpy(self.labels).to(torch.float32)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


In [77]:
# training loop
def train(train_loader, learning_rate, num_epoch, input_size):
    model = Article_nn(input_size)
    # print(train_loader)
    # loss and optimizer
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # training loop
    n_total_steps = len(train_loader)
    for epoch in range(num_epoch):
        for i, (images, labels) in enumerate(train_loader):
            sample = images.reshape(5,-1).to(device)
            labels = labels.view(labels.shape[0], 1).to(device) # makes it a column vector

            # forward
            output = model(sample)
            loss = criterion(output, labels)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i) % 1 == 0:
                print(
                    f'epoch {epoch} / {num_epoch-1}, step {i}/{n_total_steps-1} loss = {loss.item():.4f}')

    print(f"\n#################################\n# TEST DONE\n#################################\n")

    return model

In [86]:
def test(model, test_loader):
    with torch.no_grad():
        n_samples = 0
        n_diff = 0
        for images, labels in test_loader:
            sample = images.reshape(5, -1).to(device)
            labels = labels.view(labels.shape[0], 1).to(device)

            outputs = model(sample) # trained model
            n_diff += torch.mean(torch.abs(outputs-labels))
            n_samples += labels.shape[0]
            #n_correct += (pred == labels).sum().item()

        acc = n_diff/n_samples
        print(f"accuracy = {acc}")

In [87]:
if __name__ == '__main__':
    xy_len = 100
    batch_size = 5
    realisations = 20
    train_data = NumbersDataset(realisations, xy_len)
    test_data = NumbersDataset(realisations, xy_len)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    # train
    model_trained = train(train_loader, 0.5, 20, input_size)

    # test
    test(model_trained, test_loader)

epoch 0 / 19, step 0/3 loss = 0.8255
epoch 0 / 19, step 1/3 loss = 1.2399
epoch 0 / 19, step 2/3 loss = 0.8707
epoch 0 / 19, step 3/3 loss = 21.0264
epoch 1 / 19, step 0/3 loss = 12.4032
epoch 1 / 19, step 1/3 loss = 0.7559
epoch 1 / 19, step 2/3 loss = 21.1135
epoch 1 / 19, step 3/3 loss = 27.5382
epoch 2 / 19, step 0/3 loss = 14.1555
epoch 2 / 19, step 1/3 loss = 38.2865
epoch 2 / 19, step 2/3 loss = 5.7703
epoch 2 / 19, step 3/3 loss = 21.1354
epoch 3 / 19, step 0/3 loss = 20.3003
epoch 3 / 19, step 1/3 loss = 25.1607
epoch 3 / 19, step 2/3 loss = 25.5115
epoch 3 / 19, step 3/3 loss = 57.9778
epoch 4 / 19, step 0/3 loss = 54.8518
epoch 4 / 19, step 1/3 loss = 27.2165
epoch 4 / 19, step 2/3 loss = 12.1948
epoch 4 / 19, step 3/3 loss = 41.5971
epoch 5 / 19, step 0/3 loss = 30.4128
epoch 5 / 19, step 1/3 loss = 26.1071
epoch 5 / 19, step 2/3 loss = 47.7720
epoch 5 / 19, step 3/3 loss = -2.2043
epoch 6 / 19, step 0/3 loss = 23.9135
epoch 6 / 19, step 1/3 loss = 48.9278
epoch 6 / 19, ste

In [33]:
len(dataloader)

4